# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup

import array      
import csv

# Jupyter Gmaps
# https://jupyter-gmaps.readthedocs.io/en/latest/install.html
import gmaps



# ipywidgets ( jupyter-widgets )
# https://ipywidgets.readthedocs.io/en/latest/user_install.html
import ipywidgets as widgets

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
from ipywidgets import Button
from ipywidgets import interact

from datetime import date

# Import API key
from api_key_file import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:

csvpath = os.path.join('../output_data','cities.csv' )
csvpath
# CSV reader specifies delimiter and variable that holds contents
    
csvreader = csv.reader(csvpath, delimiter=',')

    # Read the header row first (skip this step if there is row header)
csv_header = next(csvreader)
    
    # Read each row of data after the header
#for row in csvreader:        
       # print(row) 
cities_df = pd.read_csv(csvpath)
cities_df       

,name,coord.lat,coord.lon,main.temp_max,main.humidity,clouds.all,wind.speed,sys.country,dt
0,Husavik,66.0449,-17.3389,270.10,81,100,1.77,IS,1640493257
1,Jamestown,42.0970,-79.2353,278.77,95,90,2.24,US,1640492794
2,Albany,42.6001,-73.9662,278.47,95,100,1.79,US,1640492988
3,Zunyi,27.6867,106.9072,273.20,95,100,3.69,CN,1640493259
4,Ushuaia,-54.8000,-68.3000,282.96,62,40,3.09,AR,1640493259
...,...,...,...,...,...,...,...,...,...
141,Alofi,-19.0595,-169.9187,301.09,69,54,2.57,NU,1640496458
142,Castro,-24.7911,-50.0119,288.93,57,35,1.41,BR,1640496459
143,Kirtipur,27.6667,85.2833,286.11,62,20,1.03,NP,1640496460
144,Half Moon Bay,37.4636,-122.4286,281.62,89,75,2.24,US,1640496460


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.
# with open(csvpath) as csvfile:

      

In [21]:

       gmaps.configure(api_key=g_key)

locations = cities_df[['coord.lat', 'coord.lon']]
weights = cities_df['main.humidity']

fig = gmaps.figure(map_type='ROADMAP', center=(51.5, -0.2), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 10

fig.add_layer(heatmap_layer)
fig 



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
filtered_city_data = cities_df[(cities_df['main.temp_max']>250) & (cities_df['main.temp_max']<275) 
                               & (cities_df['clouds.all']==0) & (cities_df['wind.speed']<10)]
filtered_city_data.dropna

filtered_city_data


,name,coord.lat,coord.lon,main.temp_max,main.humidity,clouds.all,wind.speed,sys.country,dt
18,Tasiilaq,65.6145,-37.6368,267.24,56,0,3.02,GL,1640493269
119,Zhangye,38.9342,100.4517,266.20,40,0,1.25,CN,1640493466
121,Fengcheng,40.4536,124.0717,263.40,26,0,3.59,CN,1640493468


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = filtered_city_data[['name', 'sys.country', 'coord.lat', 'coord.lon']]

hotel_name_arr = []
hotel_lat_arr = []
hotel_lng_arr = []

for index, row in hotel_df.iterrows():  
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="    
    url += str(row['coord.lat']) + "%2C" + str(row['coord.lon']) + "&radius=5000&type=lodging&key=" + g_key    
    # print(url)    
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    if(response.status_code == 200):
        json_resp = response.json()
        # print(json_resp) 

        results = json_resp['results']
        if((not(results is None)) & (len(results) > 0)):        
            hotel_name = json_resp['results'][0]['name']
            hotel_name_arr.append(hotel_name)

            hotel_lat = json_resp['results'][0]['geometry']['location']['lat']
            hotel_lat_arr.append(hotel_lat)

            hotel_lng = json_resp['results'][0]['geometry']['location']['lng']
            hotel_lng_arr.append(hotel_lng)    

hotel_df.insert(4, "Hotel Name", hotel_name_arr, True)
hotel_df.insert(5, "Hotel Lat", hotel_lat_arr, True)
hotel_df.insert(6, "Hotel Lng", hotel_lng_arr, True)

hotel_df

,name,sys.country,coord.lat,coord.lon,Hotel Name,Hotel Lat,Hotel Lng
18,Tasiilaq,GL,65.6145,-37.6368,The Red House,65.610278,-37.624167
119,Zhangye,CN,38.9342,100.4517,Zhangye Lvzhou Holiday Inn,38.933208,100.428128
121,Fengcheng,CN,40.4536,124.0717,Hanting Hotel Fengcheng Phoenix Station,40.449456,124.062290


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["coord.lat", "coord.lon"]]

KeyError: 'City'

In [31]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

NameError: name 'hotel_locations' is not defined